**This notebook is covered by the following [license](../License.ipynb)  This note must not be removed**

# Imagetragick Creator

- from https://github.com/mike-williams/imagetragick-poc
- using OpenShift imagestreams and builds https://www.tutorialworks.com/openshift-dockerfile/

In [2]:
crc status

CRC VM:          Running
OpenShift:       Running (v4.11.0)
Podman:          
Disk Usage:      14.94GB of 32.74GB (Inside the CRC VM)
Cache Usage:     96.58GB
Cache Directory: /home/thomas/.crc/cache


In [3]:
crc console --credentials

To login as a regular user, run 'oc login -u developer -p developer https://api.crc.testing:6443'.
To login as an admin, run 'oc login -u kubeadmin -p RRLE6-s7q2X-jVyYU-xoUog https://api.crc.testing:6443'


In [4]:
oc new-project imagetragick

Now using project "imagetragick" on server "https://api.crc.testing:6443".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



In [20]:
oc create imagestream poc

imagestream.image.openshift.io/poc created


In [22]:
oc create -f - <<EOF
kind: BuildConfig
apiVersion: build.openshift.io/v1
metadata:
  name: poc
spec:
  strategy:
    type: Docker
    dockerStrategy:
      dockerfilePath: Dockerfile
  source:
    type: Git
    git:
      uri: 'https://github.com/thomasfricke/docker-imagetragick'
      ref: python3
  triggers:
    - type: ConfigChange
  runPolicy: Serial
  output:
    to:
      kind: ImageStreamTag
      name: poc:latest
EOF

buildconfig.build.openshift.io/poc created


In [26]:
oc start-build poc --follow

build.build.openshift.io/poc-4 started
Cloning "https://github.com/thomasfricke/docker-imagetragick" ...
	Commit:	9c8659e6369e4255e897f6b95e7f9144f8988544 (removed uploads and linked it to /tmp)
	Author:	Thomas Fricke <github@thomasfricke.de>
	Date:	Fri Aug 19 12:10:37 2022 +0200
time="2022-08-18T08:25:25Z" level=info msg="Not using native diff for overlay, this may cause degraded performance for building images: kernel has CONFIG_OVERLAY_FS_REDIRECT_DIR enabled"
I0818 08:25:25.259364       1 defaults.go:102] Defaulting to storage driver "overlay" with options [mountopt=metacopy=on].
Caching blobs under "/var/cache/blobs".

Pulling image centos:7 ...
Resolving "centos" using unqualified-search registries (/etc/containers/registries.conf)
Trying to pull registry.redhat.io/centos:7...
time="2022-08-18T08:25:26Z" level=warning msg="Failed, retrying in 1s ... (1/3). Error: initializing source docker://registry.redhat.io/centos:7: reading manifest 7 in registry.redhat.io/centos: unknown: No

In [27]:
oc get all

NAME                       READY   STATUS       RESTARTS   AGE
pod/poc-1-build            0/1     Init:Error   0          46m
pod/poc-2-build            0/1     Init:Error   0          46m
pod/poc-3-build            0/1     Completed    0          40m
pod/poc-4-build            0/1     Completed    0          114s
pod/poc-776cb8bc9c-pksz5   1/1     Running      0          38m

NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/poc   ClusterIP   10.217.4.167   <none>        8080/TCP   36m

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/poc   1/1     1            1           38m

NAME                             DESIRED   CURRENT   READY   AGE
replicaset.apps/poc-776cb8bc9c   1         1         1       38m

NAME                                 TYPE     FROM          LATEST
buildconfig.build.openshift.io/poc   Docker   Git@python3   4

NAME                             TYPE     FROM          STATUS                       STARTED       

In [15]:
oc logs poc-2-build -f

Defaulted container "docker-build" out of: docker-build, git-clone (init), manage-dockerfile (init)
time="2022-08-18T06:03:55Z" level=info msg="Not using native diff for overlay, this may cause degraded performance for building images: kernel has CONFIG_OVERLAY_FS_REDIRECT_DIR enabled"
I0818 06:03:55.926945       1 defaults.go:102] Defaulting to storage driver "overlay" with options [mountopt=metacopy=on].
Caching blobs under "/var/cache/blobs".

Pulling image centos:7 ...
Resolving "centos" using unqualified-search registries (/etc/containers/registries.conf)
Trying to pull registry.redhat.io/centos:7...
time="2022-08-18T06:03:56Z" level=warning msg="Failed, retrying in 1s ... (1/3). Error: initializing source docker://registry.redhat.io/centos:7: reading manifest 7 in registry.redhat.io/centos: unknown: Not Found"
time="2022-08-18T06:03:58Z" level=warning msg="Failed, retrying in 2s ... (2/3). Error: initializing source docker://registry.redhat.io/centos:7: reading manifest 7 in regi